In [ ]:
import random
import numpy as np
import glob

#from tensorflow import keras
#from mlp_mixer import MlpMixerModel

from skimage.io import imread, imread_collection, imshow, imsave
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    cohen_kappa_score,
)


In [ ]:
images_bacillus = glob.glob("dataset_mlp_200x200/bacilo/train/images/*.png")
images_no_bacillus = glob.glob("dataset_mlp_200x200/no_bacilo/train/images/*.png")


In [ ]:


def load_data(bac, no_bac, tam_img):
    
    images_bacillus = glob.glob(bac)
    images_bacillus = random.sample(images_bacillus, tam_img)
    images_no_bacillus = glob.glob(no_bac)
    images_no_bacillus = random.sample(images_no_bacillus, len(images_bacillus))
    images = images_bacillus + images_no_bacillus
    labels = np.concatenate((np.ones(len(images_bacillus)), np.zeros(len(images_no_bacillus))))
    images = imread_collection(images)
    images = list(map(lambda x: np.asarray(x), images))
    labels = np.asarray(labels).astype('float32').reshape((-1,1))
    
    return images, labels

In [ ]:
x_train, y_train = load_data("dataset_mlp_200x200/bacilo/train/images/*.png", "dataset_mlp_200x200/no_bacilo/train/images/*.png", 50)
x_val, y_val = load_data("dataset_mlp_200x200/bacilo/test/images/*.png", "dataset_mlp_200x200/no_bacilo/test/images/*.png", 50)

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_val, y_val = np.array(x_val), np.array(y_val)

In [ ]:
from mlp_mixer import MlpMixerModel

In [ ]:
model_1 = MlpMixerModel(input_shape=x_train.shape[1:],
                      num_classes=len(np.unique(y_train)),
                      num_blocks=4, 
                      patch_size=1,
                      hidden_dim=32, 
                      tokens_mlp_dim=64,
                      channels_mlp_dim=128,
                      use_softmax=True
                      )
                      
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model_1.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=30)

In [ ]:
model_1.save_weights("model.h5")